In [1]:
import os 
import json

In [2]:
def digest(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #df = pd.read_json(file, lines=True)
    return data


In [3]:
entries = [digest(entry) for entry in os.scandir('../../data/') if entry.is_file()]

In [4]:
entries[0][2]['features'][4]['properties']


{'trip': {'cis_agency_name': 'ABOUT ME',
  'cis_id': '100166',
  'cis_number': 96,
  'cis_order': 5,
  'cis_parent_route_name': '166',
  'cis_real_agency_name': 'ABOUT ME',
  'cis_short_name': '166',
  'cis_vehicle_registration_number': 1935,
  'gtfs_route_id': 'L166',
  'gtfs_route_short_name': '166',
  'gtfs_trip_id': '166_292_181001',
  'id': '2019-08-12T19:09:00Z_100166_166_96',
  'start_cis_stop_id': 59501,
  'start_cis_stop_platform_code': 'B',
  'start_time': '21:09:00',
  'start_timestamp': '2019-08-12T19:09:00.000Z',
  'vehicle_type': 3,
  'wheelchair_accessible': True},
 'last_position': {'bearing': 75,
  'cis_last_stop_id': 59395,
  'cis_last_stop_sequence': 33,
  'delay': 269,
  'delay_stop_arrival': None,
  'delay_stop_departure': 284,
  'gtfs_last_stop_id': None,
  'gtfs_last_stop_sequence': None,
  'gtfs_next_stop_id': 'U63Z1P',
  'gtfs_next_stop_sequence': None,
  'gtfs_shape_dist_traveled': '16.8',
  'is_canceled': False,
  'lat': '50.15097',
  'lng': '14.51525',
  'or

In [5]:
import psycopg2

In [15]:
conn = psycopg2.connect("host=localhost port=5432 dbname=pid-portal user=postgres password=passwd")

In [16]:
cur = conn.cursor()

In [17]:
cur.execute('SELECT * FROM "User"')

In [18]:
cur.fetchone()

(1, 'test', 'test')

In [21]:
i = 0
for line in entries[0]:
    for data in line['features']:
        cur.execute("""
                INSERT INTO "Vehicle_position" (vehicle_id, timestamp, bearing, calculated_delay, speed, longitude, latitude)
                VALUES (%s, %s, %s, %s, %s, %s, %s);
                """,
               (data['properties']['trip']['gtfs_route_id'],data['properties']['trip']['start_timestamp'], 
                data['properties']['last_position']['bearing'], data['properties']['last_position']['delay'],
                data['properties']['last_position']['speed'], data['properties']['last_position']['lng'], data['properties']['last_position']['lat']))
        i += 1

In [22]:
conn.commit()